<a href="https://colab.research.google.com/github/fmuranda/Machine_Learning-/blob/main/Validation_and_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Validation and testing

In [ ]:
#@title Import libraries and dataset
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import models
from torch import optim
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
#@title Define neural network,_init_ and forward functions

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
#@title Instantiate the model
net = Net()

In [ ]:
#@title Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#@title Load and transform the data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = (0.4914, 0.4822, 0.4465),
        std = (0.2023, 0.1994, 0.2010))
])

train_dataset = CIFAR10(root='./data', train=True,
                       download=True, transform=train_transforms)

train_set, val_set = torch.utils.data.random_split(train_dataset, [40000, 10000])

trainloader = torch.utils.data.DataLoader(
                    train_set,
                    batch_size=4,
                    shuffle=True)

valloader = torch.utils.data.DataLoader(
                    val_set,
                    batch_size=4,
                    shuffle=False)

trainloader = torch.utils.data.DataLoader(
                    train_set,
                    batch_size=4,
                    shuffle=True,
                    num_workers=2)

100%|██████████| 170M/170M [00:01<00:00, 105MB/s]


In [ ]:
#@title Set the model to training mode and evaluation mode for validation
for epoch in range(10):
    net.train() # Set the model to training mode
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    net.eval() # Set the model to evaluation mode for validation
    validation_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader: # Changed testloader to valloader
          images, labels = data
          outputs = net(images)
          loss = criterion(outputs, labels)
          validation_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(trainloader)}, Validation Loss: {validation_loss/len(valloader)}") # Completed the print statement

Epoch 1, Training Loss: 1.7966558238506316, Validation Loss: 1.571040722990036
Epoch 2, Training Loss: 1.5197837747365237, Validation Loss: 1.4208078810513018
Epoch 3, Training Loss: 1.4236456586308777, Validation Loss: 1.4158243239104749
Epoch 4, Training Loss: 1.3661673892620951, Validation Loss: 1.3466906572818755
Epoch 5, Training Loss: 1.327568440438807, Validation Loss: 1.3122045918643475
Epoch 6, Training Loss: 1.2957131280086935, Validation Loss: 1.3080197643905878
Epoch 7, Training Loss: 1.2782197581339627, Validation Loss: 1.2554400429069996
Epoch 8, Training Loss: 1.251092935793288, Validation Loss: 1.288907200974226
Epoch 9, Training Loss: 1.232424367008172, Validation Loss: 1.2306092447448522
Epoch 10, Training Loss: 1.2203342051234096, Validation Loss: 1.2030589907258749
